# Web scraping pagina walmart nicaragua


In [ ]:
#install.packages("openxlsx")
library(rvest)
library(openxlsx)

In [ ]:

rm(list=ls())
url_principal="https://www.walmart.com.ni"

datos = data.frame(codigo=character(),
                  producto=character(), 
                  precio =double(),
                  precio_promocional= double(),
                  grupo=character(),
                  categoria=character(),
                  subcategoria=character(),
                  descripcion=character(),
                  imagen=character(),
                  url_articulo=character(),
                  pagina= double(),
                  articulo=double()
                )

In [ ]:
## grupos
url <- "https://www.walmart.com.ni/"
web <- read_html(url)
# grupos ----
css =".vtex-flex-layout-0-x-flexRow--rowslidecategory"
grupo=html_elements(web,css)%>% html_text()

grupo= c("abarrotes",
         "jugos-y-bebidas",
         "lacteos",
         "higiene-y-belleza",
         "limpieza-y-mascotas",
         "embutidos",
         "frutas-y-verduras",
         "carnes-y-pescados",
         "bebes-y-ninos",
         "cervezas-vinos-y-licores",
         "alimentos-congelados",
         "panaderia-y-tortilleria",
         "electronica",
         "articulos-para-el-hogar",
         "farmacia",
         "juguetes",
         "deportes"
)


[1] 17

In [ ]:
# funcion para descargar datos de cada producto

producto<- function(url,p=1,i=1){
    
    url <- url
    
    #url="https://www.walmart.com.ni/boquitas-milpa-real-tostadas-360gr/p"
    #url = "https://www.walmart.com.ni/bicicleta-next-de-nino-12/p"
    #url = "https://www.walmart.com.ni/bebida-nutri-lety-lactea-1000ml/p"
    
    web <- rvest::read_html(url)
    #codigo ----
    css<- ".vtex-product-identifier-0-x-product-identifier__value"
    codigo <-html_elements(web,css) %>% html_text2()
    
    # producto ----
    css=".vtex-store-components-3-x-productBrand"
    producto <-html_elements(web,css) %>% html_text2()
    
    # precio----
    css=".vtex-store-components-3-x-sellingPrice"
    precio <- html_element(web,css) %>% html_text2()
    precio <- gsub("C\\$","",precio) 
    precio <-gsub(",","",precio) %>% as.numeric()
    ###
    css=".vtex-store-components-3-x-listPrice"
    precio_promo <- html_element(web,css) %>% html_text2()
    precio_promo <- gsub("C\\$","",precio_promo)
    precio_promo <- gsub(",","",precio_promo) %>% as.numeric()
    ##imagen ----
    selector =".vtex-store-components-3-x-productImageTag"
    imagen  = html_element(web,selector) %>% html_attr("src")
    
    # categorias ----
    selector =".vtex-breadcrumb-1-x-arrow--1"
    categoria_g  = html_elements(web,selector) %>% html_text2()
    
    selector =".vtex-breadcrumb-1-x-arrow--2"
    categoria  = html_elements(web,selector) %>% html_text2()
    
    selector =".vtex-breadcrumb-1-x-arrow--3"
    sub_categoria  = html_elements(web,selector) %>% html_text2()
    
    # descripcion ----
    css =".vtex-store-components-3-x-content"
    descripcion=html_elements(web,css)[2] %>% html_text() %>% paste0()

    data.frame(
      codigo = codigo,
      producto = producto, 
      precio = precio,
      precio_promocional=precio_promo,
      grupo= categoria_g,
      categoria=categoria,
      subcategoria= sub_categoria,
      descripcion=descripcion,
      imagen=imagen,
      url_articulo=url,
      pagina=p,
      articulo=i
    )
    
}


In [ ]:
# descargar los datos 
# extraemos los link de cada grupo , y calculamos la cantidad de paginas a recorrer
# extraemos lista de link de los productos de cada pagina
# descargamos los datos recorriendo los link de todos los productos
# agregamos los datos al data frame de datos

for (g in 1:length(grupo)){
    try({
      ######
      url= paste0(url_principal,"/",grupo[g])
      
      web <- read_html(url)
      ### cantidad de paginas
      css =".vtex-search-result-3-x-totalProducts--layout"
      paginas=html_elements(web,css)%>% html_text() 
      paginas=round(as.numeric(gsub("  Productos","",paginas))/10,digits = 0)+1
      
      paginas = 1
      # links de cada pagina
      for (p in 1:paginas){
        try({
          url <- paste0(url_principal,"/",grupo[g],"?page=",p)
          web <- rvest::read_html(url)
          
          css_selector="a.vtex-product-summary-2-x-clearLink"
          links = html_nodes(web,css_selector) %>% html_attr("href")
          
          # descarga dato producto
          for (i in 1:length(links)) {
            url <- paste0(url_principal,links[i])
            # con la funcion producto
            datos=rbind(datos,producto(url=url,p=p,i=i))
          }
          #Sys.sleep(1)
        })
      }
    })
}

In [ ]:
# guaramos el archivo en formato excel
options(spicen=999)
write.xlsx(datos, "productos_walmart_.xlsx")
